In [1]:
"""
TBD need to know if position risk has been reduced, otherwise don't generate orders for symbol

Option 2:
    - Store initial_size and current_size of position
    - if current_size is some criteria below the initial size, then risk is reduced
    - filter out stocks where criteria is not met

Position Table Generation:
1. Each row in the position table shall obtain the following info:
    - Symbol name
    - initial size
    - entry date?
    - direction?
    -

Order Table Generation:
1.) Order Table generation shall follow this procedure:
    1. Retrieve data for current open positions
    2. For each open position:
        1. if at least one of the following is true:
            TRAIL_STOP || STOP || TARGET_EXIT || FRENCH_STOP || REGIME_END
            1. Generate an order entry, where type is the first discovered reason for entry creation
    3. For each symbol, create order entry if:
        symbol not in order_table && risk_reduced is True
    4.


Order Table Output:
1.) The following data shall be stored in the order_table, sorted by close type first. It may be a view of a larger table:
    Symbol,
    direction of trade,
    position size (dollars or shares),
    type: [
        ENTRY,
        TRAIL_STOP,
        STOP,
        TARGET_EXIT,
        FRENCH_STOP,
        REGIME_END
    ]


2.) Output of the order table shall be sorted by type, with the following priority:
    1. STOP
    2. TRAIL_STOP
    3. TARGET_EXIT
    4. FRENCH_STOP
    5. REGIME_END
    6. ENTRY



2.)
Entries/Exits in the order table shall be generated from
"""
import json
import scripts.account_data as sad
import tda_access.access as taa
import pandas as pd
import pathlib

In [2]:
td_client = taa.TdBrokerClient.init_from_json('..\\data_args\\credentials.json')

In [6]:

# init client and tables
regime_df = pd.read_excel(pathlib.Path('..') / 'data' / 'regimes.xlsx')

with open('scanner_config.json') as fp:
    scanner_args = json.load(fp)

(
    website,
    start,
    end,
    vlty_n,
    dist_pct,
    retrace_pct,
    threshold,
    dgt,
    d_vol,
    r_vol,
    lvl,
    bm_ticker,
    bm_col,
    ccy_col,
    window,
    st,
    lt,
    save_ticker_df,
    save_last_row_df,
    save_regime_df
) = list(scanner_args.values())
fast = st
slow = lt
regime_cols = [
    'fc','fc_r',
    'sma','sma_r', 'bo',
    'bo_r','tt','tt_r'
]
groupby_cols = ['score', 'delta'] + regime_cols
sort_key = ['GICS Sub-Industry']
grouped = regime_df.groupby(sort_key)[groupby_cols].mean().sort_values(
    by= ['GICS Sub-Industry','score'])

# delta is the diff between absolute and relative regime scores
delta_bound = 0.5
sub_sectors = list(grouped.loc[
    ((grouped.delta >= delta_bound) & (grouped.fc_r > 0)) |
    ((grouped.delta <= -delta_bound) & (grouped.fc_r < 0))
].index)
watch_stocks = regime_df.loc[
    (regime_df['GICS Sub-Industry'].isin(sub_sectors)) &
    (regime_df['fc_r'] * grouped['delta'].loc[regime_df['GICS Sub-Industry']].values > 0)
    , 'symbol'
]


In [7]:
import scripts

_strategy_lookup, tx_tables, price_histories, bench = sad.main(
    td_client=td_client,
    sub_sectors= [],
    symbols=['FRC'] + list(watch_stocks)
)
# inputs
equity = 25000
ticks = ['SPY']

def get_strategy_data(symbol, conn):
    price_data = pd.read_sql(
        f"SELECT * FROM price_data where symbol = \'{symbol}\'",
        conn
    )
    peak_data = pd.read_sql(
        f"SELECT * FROM peak where symbol = \'{symbol}\'",
        conn
    )
    regime_data = pd.read_sql(
        f"SELECT * FROM regime where symbol = \'{symbol}\'",
        conn
    )
    return price_data, peak_data, regime_data

def generate_order_book(conn, equity, ticks):
    order_book = pd.DataFrame()
    for tick in ticks:
        price_data, peak_data, regime_data = get_strategy_data(tick, conn)
        if regime_data.loc[regime_data['type'] == 'fc'].empty:
            continue
        stop_level, entry_price = scripts.account_data.get_stop_level(FcStrategyData())
        order_book = order_book.append(
            conn.get_order_book(tick, equity)
        )
    return order_book

HTTPStatusError: Client error '400 Bad Request' for url 'https://api.tdameritrade.com/v1/oauth2/token'
For more information check: https://httpstatuses.com/400

In [ ]:
order_table = pd.read_excel(pathlib.Path('..') / 'data' / 'order_table.xlsx', index_col=0)

In [ ]:
order_table

In [ ]:
symbol = 'BALL'
od = order_table[order_table.symbol == symbol]

In [ ]:
epd = _strategy_lookup[symbol].enhanced_price_data.copy()

In [ ]:
_pt = _strategy_lookup[symbol].peak_table.copy()
# _pt = pda.PeakTable(_pt)
# _pt = _pt.unpivot(_symbol_data.index)
# _pt = _pt.loc[~_pt.index.duplicated(keep='first')]
def filter_duplicates(_sw):
    return _sw.loc[~_sw.index.duplicated(keep='first')]

_shi_px = _pt.loc[(_pt.type == -1)].set_index('end')
_slo_px = _pt.loc[(_pt.type == 1)].set_index('end')
_shis = []
_slos = []
for i in range(1, 5):
    _shis.append(
        _shi_px.loc[(_shi_px.lvl == i)]
    )
    _shis[i-1] = filter_duplicates(_shis[i-1])
    _slos.append(
        _slo_px.loc[(_slo_px.lvl == i)]
    )
    _slos[i-1] = filter_duplicates(_slos[i-1])
    epd[f'hi{i}_lag'] = _shis[i-1].en_px
    epd[f'lo{i}_lag'] = _slos[i-1].en_px

epd['hi_band'] = epd.close.rolling(256).max()
epd['lo_band'] = epd.close.rolling(256).min()
epd['trade_range'] = epd.hi_band - epd.lo_band
epd['40pct_band'] = epd.trade_range * .4 + epd.lo_band
epd['61pct_band'] = epd.trade_range * .61 + epd.lo_band

In [ ]:
def plot(lvl):
    epd[[
        'close',
        'rg',
        f'hi{lvl}',
        f'lo{lvl}',
        f'hi{lvl}_lag',
        f'lo{lvl}_lag',
        '61pct_band',
        '40pct_band'
    ]].plot(
        style=[
            'b',
            'y',
            'ro',
            'go',
            'kv',
            'k^',
            'r-',
            'g-'
        ], figsize=[12, 5], secondary_y='rg', grid=True, title=f'symbol lvl {lvl}')

# plot(4)
plot(3)
plot(2)
plot(1)